In [1]:
import dgl
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import dgl.nn as dglnn

In [2]:
# 创建示例图数据，每个图代表一个日期
num_nodes = 5
num_dates = 10
num_features_per_node = 3  # 固定每个节点的特征数量

graph_list = []

for date in range(num_dates):
    g = dgl.DGLGraph()
    g.add_nodes(num_nodes)
    # 添加边，表示节点之间的关系
    # 这里使用随机生成的边，实际数据应根据您的数据来构建
    src = np.random.randint(0, num_nodes, size=num_nodes)
    dst = np.random.randint(0, num_nodes, size=num_nodes)
    g.add_edges(src, dst)

    # 生成随机节点特征
    node_features = torch.randn(num_nodes, num_features_per_node)
    g.ndata['features'] = node_features

    g = dgl.add_self_loop(g)  # 添加自环
    
    graph_list.append(g)

c:\Users\11622\.conda\envs\ML_project\lib\site-packages\dgl\heterograph.py:92: DGLWarning: Recommend creating graphs by `dgl.graph(data)` instead of `dgl.DGLGraph(data)`.
  dgl_warning(


In [4]:
for g in graph_list:
    print(g)
# 将多个日期的图数据合并为一个图
combined_graph = dgl.batch(graph_list)

# 将多个日期的节点特征合并为一个张量
combined_features = torch.stack([g.ndata['features'] for g in graph_list], dim=0)

print(combined_graph)
print(combined_features.shape)

Graph(num_nodes=5, num_edges=10,
      ndata_schemes={'features': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={})
Graph(num_nodes=5, num_edges=10,
      ndata_schemes={'features': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={})
Graph(num_nodes=5, num_edges=10,
      ndata_schemes={'features': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={})
Graph(num_nodes=5, num_edges=10,
      ndata_schemes={'features': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={})
Graph(num_nodes=5, num_edges=10,
      ndata_schemes={'features': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={})
Graph(num_nodes=5, num_edges=10,
      ndata_schemes={'features': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={})
Graph(num_nodes=5, num_edges=10,
      ndata_schemes={'features': Scheme(shape=(3,), dtype=torch.float32)}
      edata_schemes={})
Graph(num_nodes=5, num_edges=10,
      ndata_schemes={'features': Scheme(shape=(3,)

In [5]:
# 创建一个简单的GAT模型
class GATModel(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super(GATModel, self).__init__()
        self.gat_layer = dglnn.GATConv(in_dim, out_dim, num_heads=1)  # 使用一个注意头
        self.fc = nn.Linear(hidden_dim, out_dim)

    def forward(self, g, features):
        x = features
        x = self.gat_layer(g, x)
        x = torch.relu(x)
        x = self.fc(x)
        return x

# 创建一个简单的LSTM模型
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, (h_n, _) = self.lstm(x)
        output = self.fc(h_n[-1, :, :])  # 取LSTM最后一个时间步的输出作为预测结果
        return output

# 创建结合GAT和LSTM的模型
class GATLSTMModel(nn.Module):
    def __init__(self, gat_input_size, gat_hidden_size, gat_output_size, lstm_input_size, lstm_hidden_size, lstm_output_size):
        super(GATLSTMModel, self).__init__()
        self.gat_model = GATModel(gat_input_size, gat_hidden_size, gat_output_size)
        self.lstm_model = LSTMModel(lstm_input_size, lstm_hidden_size, lstm_output_size)

    def forward(self, g, features):
        # 使用GAT模型进行图节点特征的提取
        gat_output = self.gat_model(g, features)
        # 将GAT的输出作为LSTM的输入
        lstm_input = gat_output.unsqueeze(0)  # 添加时间步的维度
        # 使用LSTM模型进行时序预测
        lstm_output = self.lstm_model(lstm_input)
        return lstm_output

In [6]:
# 初始化模型
gat_input_size = num_features_per_node  # 节点特征的维度
gat_hidden_size = 16
gat_output_size = 1
lstm_input_size = gat_output_size  # LSTM的输入维度与GAT的输出维度相同
lstm_hidden_size = 16
lstm_output_size = 1

model = GATLSTMModel(gat_input_size, gat_hidden_size, gat_output_size, lstm_input_size, lstm_hidden_size, lstm_output_size)

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [10]:
# 将数据转换为PyTorch张量
features_tensor = combined_features.unsqueeze(-1)  # 添加维度，使其成为三维张量
labels = torch.randn(num_dates, lstm_output_size)  # 随机生成标签，实际应根据数据替换

print(features_tensor.shape)
print(labels.shape)

torch.Size([10, 5, 3])
torch.Size([10, 1])


In [11]:
# 训练模型
num_epochs = 100

for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(combined_graph, features_tensor)  # 使用多个日期的数据
    loss = criterion(output, labels)
    loss.backward()
    optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}] Loss: {loss.item()}")

# 使用模型进行时序预测
# ...


DGLError: Expect number of features to match number of nodes (len(u)). Got 10 and 50 instead.